<!-- <html>
    <style>
    .rendered_html p {
        font-size: 17px;
        font-family: Times;
        text-align: left}
    </style>
</html> -->
<style>
.text_cell_render {
    font-family: "Times New Roman", serif;
}
</style>

<div style="display:none">
<style>
    .text_cell_render {
        font-family: "Times New Roman", serif;
    }
    body {
      font-family: "Times New Roman", Times, serif !important;
    }
</style>
</div>


<!-- Logo -->
<div style="margin: 24px 0 56px 0;">
  <img src="static/images/harvard-dce-logo.svg"
       alt="Harvard Division of Continuing Education"
       style="height:64px;">
</div>

<!-- Program / Degree -->
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 42px;
  font-weight: 400;
  line-height: 1.15;
  margin: 0 0 56px 0;
">
  Applied Quantitative Finance and Machine Learning
</div>

<!-- Paper Title -->
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 48px;
  font-weight: 400;
  line-height: 1.2;
  margin: 0 0 24px 0;
">
  Scaling Machine Learning Signals<br>for Equity Factor Portfolios
</div>

<!-- Subtitle -->
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
  line-height: 1.5;
  margin: 0 0 40px 0;
">
  An applied study using 35 years of Fama–French 100 portfolios at daily frequency
</div>

<!-- Author / Date -->
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 26px;
  font-weight: 400;
  line-height: 1.6;
  margin: 0 0 24px 0;
">
  Final Project — Colin Alexander, CFA, CIPM<br>
  December 2025
</div>


<div id="executive-summary" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
  line-height: 1.5;
  margin: 0 0 40px 0;
">
  <strong>Executive Summary</strong>
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  /* line-height: 1.6; */
  /* margin: 0 0 24px 0; */
">
This case study examines how increasing data richness affects the behavior and usability of machine-learning signals in equity factor portfolios. Using Fama–French 100 Size×Book-to-Market portfolios at daily frequency over a 35-year window, the analysis compares linear and nonlinear models under a leakage-aware walk-forward framework, evaluating both cross-sectional ranking performance and realized portfolio outcomes after transaction costs.
<br>
<br>
Three findings are particularly relevant for practitioners.
<br>

- First, expanding both the cross-section and the frequency of the data environment improves the stability of ranking signals relative to small, monthly portfolio panels, but does not eliminate meaningful differences across model classes. Across all specifications, daily Information Coefficients (ICs) are modest in magnitude—on the order of 0.5–1.0%—yet consistently positive, indicating economically meaningful but noisy cross-sectional structure. Importantly, richer data does not mechanically favor more flexible models.

- Second, nonlinear models capture some interaction-driven structure, but their advantages are incremental rather than transformative. Random Forests and a shallow neural network exhibit ranking performance comparable to linear baselines, while boosted trees perform worse once evaluated under the same walk-forward protocol, feature set, and target horizon. These results suggest that, in a high-frequency portfolio setting with disciplined feature construction, linear factor structure remains dominant and additional model flexibility does not guarantee stronger ranking signals.

- Third, translating signals into implementable portfolios materially changes conclusions drawn from predictive metrics alone. Models with similar or even higher ranking skill can underperform economically once turnover and transaction costs are applied. In this setting, the linear baseline combines modest but stable ranking performance with low turnover, resulting in comparatively robust net outcomes. More flexible models—including boosted trees—generate less stable rankings and higher trading intensity, leading to inferior net performance across realistic transaction cost assumptions.
<br>
Overall, the case illustrates that data scale, model choice, and portfolio implementation interact tightly in practice. Improving any single component in isolation—whether by adding data, increasing model flexibility, or optimizing predictive metrics—does not ensure usable results. Practical value depends on how signals behave when deployed under realistic constraints, where turnover, stability, and robustness often matter more than incremental gains in raw predictive accuracy.

</div>

---

<div id="section-1" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
">
  <strong>1. Case Context and Question</strong>
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">
    
Equity factor models form a core building block of systematic investing. By grouping stocks according to characteristics such as size, value, profitability, and momentum, factor portfolios provide a structured and interpretable way to study return behavior across the cross-section of equities. Traditionally, these models are evaluated using linear regressions on **monthly data**, a choice that balances noise reduction, interpretability, and statistical convenience.

At the same time, advances in machine learning have raised the possibility that **nonlinear models** can capture interaction effects and state-dependent behavior that linear factor frameworks may miss. Tree-based ensembles and neural networks, in particular, offer flexible approximations that can adapt to heterogeneous relationships across assets. However, their usefulness in asset pricing remains contested, in part because financial return data are noisy, low signal-to-noise, and sensitive to evaluation choices.

A central challenge in this debate is **data scale**. When models are applied to small portfolio panels at low frequency, it is difficult to disentangle genuine structure from estimation noise. Poor performance by more flexible models may reflect insufficient data, objective mismatch, or instability under realistic implementation, rather than fundamental limitations of the models themselves. Conversely, apparent gains in predictive metrics may fail to survive once signals are translated into portfolios and exposed to trading frictions.

This case study examines that tension by holding the historical window fixed while **scaling the data environment**. Specifically, it moves from small, monthly portfolio panels to a richer setting using **Fama–French 100 Size×Book-to-Market portfolios at daily frequency over approximately 35 years**. This expansion increases both cross-sectional breadth and temporal resolution, providing a more demanding and informative environment in which to assess model behavior.

The analysis is framed from a practitioner’s perspective. Rather than asking which model is “best” in an abstract sense, the case focuses on how different modeling choices behave when evaluated through **ranking performance and portfolio outcomes**, including turnover and transaction costs. The objective is to understand the trade-offs that emerge as data scale increases and to clarify which conclusions are robust once signals are deployed under realistic constraints.

**The central question is whether richer data meaningfully changes model behavior and portfolio outcomes.**

</div>

---

<div id="section-2" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
">
  <strong>2. Data & Experimental Setup</strong>
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">
    
This section defines the <strong>environment in which the modeling and portfolio decisions are evaluated</strong>.

The analysis uses <strong>daily returns for the Fama–French 100 Size×Book-to-Market portfolios</strong> over a roughly <strong>35-year window</strong>, matching the historical span used in standard factor research while substantially increasing both cross-sectional breadth and temporal resolution. Each portfolio represents a value-weighted basket of U.S. equities sorted independently by firm size and book-to-market characteristics, forming a 10×10 grid that spans the size–value spectrum. The underlying factor definitions and economic interpretations are summarized in <a id="ref-a" href="#appendix-a"><strong>Appendix&nbsp;A</strong></a>.

Although individual stock-level holdings are not observed, each Fama–French portfolio can be interpreted as a <strong>tradable equity basket</strong> constructed according to transparent and well-documented rules. Throughout the analysis, portfolios are treated as atomic assets with “look-through” exposure to the underlying stocks, a standard assumption in academic and applied factor studies. This framing allows signals to be ranked across portfolios and translated into long–short allocations without relying on security-level data.

Using daily rather than monthly frequency materially changes the data environment. The higher frequency increases the number of observations available for model estimation and evaluation, while also introducing additional noise and short-term volatility. At the same time, expanding from 25 to 100 portfolios increases cross-sectional dispersion, creating a richer setting in which to assess whether flexible models benefit from additional breadth or simply amplify instability.

Table 1 summarizes the key dataset dimensions, including the number of portfolios, observation count, and date range. This comparison highlights the magnitude of the scaling relative to standard monthly portfolio panels and motivates the subsequent analysis of model behavior under a richer data regime.
</div>

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  margin: 24px 0;
">

  <strong>Table&nbsp;1: Dataset dimensions and scaling</strong>

  <table style="
    width:100%;
    border-collapse: collapse;
    margin-top: 12px;
    font-size: 18px;
  ">
    <thead>
      <tr>
        <th style="border-bottom: 1px solid #000; text-align: left; padding: 6px;">
          Dataset
        </th>
        <th style="border-bottom: 1px solid #000; text-align: right; padding: 6px;">
          Frequency
        </th>
        <th style="border-bottom: 1px solid #000; text-align: right; padding: 6px;">
          Portfolios
        </th>
        <th style="border-bottom: 1px solid #000; text-align: right; padding: 6px;">
          Observations&nbsp;/&nbsp;Portfolio
        </th>
        <th style="border-bottom: 1px solid #000; text-align: right; padding: 6px;">
          Date&nbsp;Range
        </th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td style="padding: 6px;">Fama–French Size×Value (25)</td>
        <td style="padding: 6px; text-align: right;">Monthly</td>
        <td style="padding: 6px; text-align: right;">25</td>
        <td style="padding: 6px; text-align: right;">≈&nbsp;420</td>
        <td style="padding: 6px; text-align: right;">1990–2025</td>
      </tr>
      <tr>
        <td style="padding: 6px;">Fama–French Size×Value (100)</td>
        <td style="padding: 6px; text-align: right;">Daily</td>
        <td style="padding: 6px; text-align: right;">100</td>
        <td style="padding: 6px; text-align: right;">≈&nbsp;9,000</td>
        <td style="padding: 6px; text-align: right;">1990–2025</td>
      </tr>
    </tbody>
  </table>

  <div style="
    font-size: 16px;
    margin-top: 8px;
  ">
    <em>
      Notes: Observation counts are approximate and reflect trading-day availability. 
        While daily FF25 portfolios are available, the standard benchmark in the literature evaluates 
        FF25 portfolios at monthly frequency. 
        The FF100 daily panel therefore substantially expands both temporal resolution and 
        cross-sectional breadth relative to conventional monthly portfolio panels.
    </em>
  </div>

</div>



In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd

# Add project root to Python path
project_root = Path.cwd().resolve().parents[0]
sys.path.insert(0, str(project_root))

from src.harvard.plots import (
    plot_size_value_marginal_means,
    plot_size_value_returns_heatmap,
)

data_root = "../../final-project-data/data/processed/"

# Load daily FF100 portfolio returns
portfolio_returns = pd.read_parquet(
    os.path.join(data_root, "portfolio_excess_returns_100_daily.parquet")
)

portfolio_returns.set_index(["Date", "PortfolioPair"]).tail()

In [ ]:
plot_size_value_returns_heatmap(portfolio_returns)

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  line-height: 1.4;
  text-align: left;
">
  <strong>Figure 1. Cross-sectional structure of average daily returns</strong><br><br>
<strong>Figure 1</strong> summarizes the cross-sectional pattern of average daily excess returns across the
Fama–French 100 Size × Book-to-Market portfolios over the full sample period. Across the full grid, average daily excess returns range from approximately −1 bp in the smallest low book-to-market portfolio (ME01–BM01) to about 6–7 bps in higher book-to-market portfolios, with most cells clustered tightly between 3 and 5 bps per day. Rather than exhibiting a simple monotonic size or value premium, average returns display substantial heterogeneity across the grid, with the lowest mean returns concentrated in the smallest size decile (ME01) and the lowest book-to-market portfolios (BM01).
<br><br>
    The color scale is centered at zero and restricted to ±0.05% to highlight economically meaningful cross-sectional variation in average daily excess returns.
<br><br>
This pattern is not inconsistent with the canonical size and value effects documented in the
literature. Those effects are typically identified in long–short factor constructions and at
monthly horizons, whereas the present analysis considers daily portfolio-level excess returns
averaged over a long sample that includes extended periods in which small and low book-to-market
portfolios underperformed. At the daily frequency, mean returns are particularly sensitive to
crisis episodes, tail events, and prolonged drawdowns, which can materially depress unconditional
averages even when factor premia are present in relative terms.
<br><br>
Across the interior of the grid, average returns are relatively more homogeneous, while pronounced non-monotonicities emerge at the extremes of the size–value distribution. Taken together, these patterns indicate that level effects alone provide a weak guide to expected performance and that economically relevant information is embedded in relative cross-sectional dispersion rather than in unconditional means. Averaged across the value dimension, mean daily returns by size range narrowly from roughly 3.2 to 4.3 bps, while value-sorted portfolios span approximately 3.0 to 4.6 bps per day, confirming that marginal size and value effects are economically modest and non-monotonic at the daily horizon.
</div>

In [ ]:
plot_size_value_marginal_means(portfolio_returns)

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">
Taken together, these diagnostics highlight a cross-section characterized by strong dispersion and
non-monotonic tails, motivating a parsimonious, ranking-based modeling approach. Rather than
forecasting return levels directly, subsequent sections focus on whether models can reliably rank
portfolios by future relative performance in the presence of fat-tailed and heterogeneous
cross-sectional structure.
<br><br>
As documented in <strong><a id="ref-b" href="#appendix-b">Appendix&nbsp;B</a></strong>, daily excess returns across the FF100 portfolios exhibit pronounced skewness, excess kurtosis, and substantial cross-sectional heterogeneity. These distributional properties indicate that unconditional mean returns provide a limited guide to relative performance, motivating a ranking-based evaluation framework throughout the analysis.
<br><br>    
The next section describes the construction of the input features used to generate these portfolio
rankings, emphasizing a parsimonious and economically interpretable design aligned with the observed
structure of the data.

---

<div id="section-3" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
">
  <strong>3. Feature Construction</strong>
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">

Feature construction is intentionally <strong>minimal and disciplined</strong>. The objective is not to engineer a large set of predictive signals, but to isolate how <strong>data scale</strong>—cross-sectional breadth and frequency—affects model behavior under a controlled and interpretable feature set.

Predictors are drawn from three simple groups. First, the analysis includes <strong>daily factor levels</strong>, capturing contemporaneous exposures to market, size, value, profitability, investment, and momentum. Second, <strong>short- and medium-horizon lags</strong> of these factors are included at trading-day equivalents of 1, 3, 6, and 12 months (21, 63, 126, and 252 trading days), allowing models to learn basic temporal dynamics without introducing long-memory effects. Third, <strong>rolling summary statistics</strong>—including a fixed 252-day (12-month) rolling mean and volatility—provide coarse information about longer-run factor behavior.

No additional technical indicators, nonlinear transformations, or interaction features are introduced. This restraint is intentional. By limiting the feature set, the analysis avoids conflating model flexibility with feature complexity and ensures that any observed changes in signal behavior are driven primarily by the richer data environment rather than by aggressive feature engineering.

All feature transformations are constructed in a <strong>leakage-aware</strong> manner. For each training window, rolling statistics and any required scaling or normalization are computed using past data only and then applied consistently to subsequent periods. This approach preserves temporal integrity while remaining light enough for an applied setting.

The resulting feature set is compact, interpretable, and shared across all models. This design choice allows differences in outcomes to be attributed to <strong>model behavior and data scale</strong>, rather than to differences in information content. A complete glossary of daily features and construction details is provided in <strong><a id="ref-c" href="#appendix-c">Appendix&nbsp;C</a></strong>.

This design is particularly well suited to the FF100 universe, where cross-sectional dispersion and non-monotonic structure limit the usefulness of direct level prediction.
</div>

In [ ]:
# Load daily factor feature panel
factor_panel = pd.read_parquet(os.path.join(data_root, "factor_panel_daily.parquet"))

# Align on common trading dates
common_dates = sorted(set(factor_panel["Date"]).intersection(portfolio_returns["Date"]))

factor_panel = factor_panel[factor_panel["Date"].isin(common_dates)]
portfolio_returns = portfolio_returns[portfolio_returns["Date"].isin(common_dates)]

# Basic sanity checks (dimensions and date coverage)
factor_panel.head(), factor_panel.shape

In [ ]:
start, end, obs = (
    factor_panel["Date"].min(),
    factor_panel["Date"].max(),
    factor_panel["Date"].nunique(),
)
print(f"Start date: {start.date()}\nEnd date: {end.date()}\nObservations: {obs}")

In [ ]:
portfolio_returns.head(), portfolio_returns.shape

In [ ]:
# Drop rows with missing features (expected early-window NaNs)
factor_panel = factor_panel.dropna().reset_index(drop=True)

factor_panel.shape

In [ ]:
start, end, obs = (
    factor_panel["Date"].min(),
    factor_panel["Date"].max(),
    factor_panel["Date"].nunique(),
)
print(f"Start date: {start.date()}\nEnd date: {end.date()}\nObservations: {obs}")

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">With the daily feature panel constructed and aligned, the analysis proceeds to walk-forward evaluation.</div>

---

<div id="section-4" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
">
    
  **4. Walk-forward Evaluation**
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">

To assess model behavior without introducing look-ahead bias, all analysis is conducted using a **rolling walk-forward evaluation** over the full 35-year sample window. At each step, models are trained on historical data and evaluated on subsequent, unseen observations, preserving the natural time ordering of financial returns.

The walk-forward design reflects how signals would be formed and assessed in practice. Models are fit using past information only, predictions are generated for future periods, and the window then advances through time. This approach avoids reliance on static train–test splits and allows performance to be examined across a wide range of market environments.

To further prevent information leakage, a short purge buffer is inserted between each training/validation window and the subsequent test window, mitigating the impact of serial dependence arising from overlapping return horizons that could otherwise bias out-of-sample evaluation.

Evaluation focuses on **cross-sectional ranking performance** rather than level prediction. For each test period, the primary metric is the **Information Coefficient (IC)**, defined as the Spearman rank correlation between predicted and realized returns across portfolios. IC provides a robust, scale-free measure of whether a model correctly orders assets, even when absolute return forecasts are noisy.

This evaluation framework establishes a consistent basis for comparing models before introducing portfolio construction and trading frictions, yielding approximately 355 non-overlapping test windows over the 35-year sample and enabling model behavior to be assessed across a wide range of market environments. It answers a preliminary but essential question: **given only past information, does a model produce rankings that align with future relative performance?**

</div>

<img src="static/images/cv_daily.png"
     style="max-width: 900px; width: 100%; height: auto; display: block; margin-bottom: 12px;">

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 18px;
  font-weight: 400;
  line-height: 1.4;
  text-align: left;
">
  <strong>Figure 2.</strong>
  Rolling walk-forward cross-validation design translated from the 54–6–1–1 monthly schema into
  trading-day equivalents (1134–126–21–21, not to scale). Each fold trains on 1,134 trading days,
  validates on 126 trading days, skips a 21-day purge buffer to eliminate temporal overlap,
  and evaluates performance on a 21-day held-out test window. The window then advances forward
  sequentially through time, preserving temporal order and preventing information leakage.
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">
</div>

In [ ]:
from src.harvard.pipeline import make_splits

dates = factor_panel["Date"]
splits = make_splits(dates)

print("splits count:", len(splits))
print("split keys:", ", ".join(splits[0]))

In [ ]:
def fold_unique_days(split):
    tr = dates.iloc[split["train_idx"]].nunique()
    va = dates.iloc[split["val_idx"]].nunique()
    te = dates.iloc[split["test_idx"]].nunique()
    return tr, va, te


def fold_date_bounds(split):
    tr = dates.iloc[split["train_idx"]]
    va = dates.iloc[split["val_idx"]]
    te = dates.iloc[split["test_idx"]]
    return tr.max(), va.min(), va.max(), te.min(), te.max()


# Show first fold day counts
print("Fold 1 unique trading days (train, val, test):", fold_unique_days(splits[0]))

# Check strict ordering on a few folds
for k in [0, 1, 2, 10, -1]:
    train_end, val_start, val_end, test_start, test_end = fold_date_bounds(splits[k])
    ok = (train_end < val_start) and (val_end < test_start)
    print(
        f"fold {k:>3}:",
        "OK" if ok else "BAD",
        "| train_end:",
        train_end.date(),
        "| test_start:",
        test_start.date(),
        "| test_end:",
        test_end.date(),
    )

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">
<br>
With this walk-forward evaluation framework in place, the next section describes the models used to generate portfolio rankings within each walk-forward fold.
</div>

---

<div id="section-5" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
">
    
  **5. Modeling Approach**
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">

The modeling approach compares a small set of **representative linear and nonlinear methods** that are commonly used in applied quantitative finance. The goal is not to exhaust the model space, but to contrast how different levels of flexibility behave as the data environment becomes richer.

As a linear baseline, the analysis uses **Ridge regression**. Ridge preserves the interpretability of linear factor models while stabilizing estimation through regularization, making it a natural benchmark for assessing whether additional model complexity delivers incremental value.

To introduce nonlinearity, the primary nonlinear benchmark is a **Random Forest**. Random Forests capture interaction effects and conditional relationships through ensembles of decision trees while retaining robustness through averaging. In prior factor studies, tree-based models have often proven effective in low- to moderate-dimensional settings, making them a useful reference point for evaluating nonlinear structure in portfolio returns.

A **shallow neural network** is also included as a sensitivity check. Neural networks offer a more flexible function class than tree ensembles, but are known to be sensitive to data scale, noise, and objective choice in financial applications. Here, the network is intentionally kept simple and is not treated as a primary contender; instead, it provides context for how highly flexible models behave when applied to portfolio-level return prediction.

Models are trained using rolling walk-forward estimation to preserve temporal ordering and avoid look-ahead bias. Scale-sensitive models—Ridge regression and the neural network—use feature standardization fit within each training window and applied consistently to subsequent periods. All models are trained using mean-squared error for numerical stability, while evaluation focuses on **cross-sectional ranking performance**, measured by the Information Coefficient (Spearman correlation).

This mismatch between the training objective and the evaluation metric is common in applied settings and is noted here because it can influence relative model behavior, particularly for neural networks. Accordingly, neural network results are interpreted cautiously and primarily as indicators of sensitivity rather than definitive evidence of superiority. The emphasis remains on how different model classes respond to increased data scale and how their signals behave once translated into portfolios.

For completeness, <a id="ref-d" href="#appendix-d"><strong>Appendix&nbsp;D</strong></a> documents the exact model architectures (including the MLP 64–32–1 configuration), regularization settings, and training protocols used in the analysis.
</div>

In [ ]:
RECOMPUTE_MODELS = True
RUN_XGBOOST = True
predictions_path = Path("../results/harvard_predictions.parquet")
predictions_path.parent.mkdir(parents=True, exist_ok=True)

if RECOMPUTE_MODELS:
    from src.harvard.pipeline import run_walkforward_experiment_parallel

    artifacts = run_walkforward_experiment_parallel(
        panel=factor_panel,
        splits=splits,
        portfolio_col="Portfolio",
        target_col="ExcessReturn_t+21",
        max_workers=12,
        rf_n_jobs=1,
        include_xgb=RUN_XGBOOST,
    )
    predictions = artifacts.predictions
    predictions.to_parquet(predictions_path, index=False)
else:
    predictions = pd.read_parquet(predictions_path)
    expected_cols = {
        "Date",
        "PortfolioPair",
        "Actual",
        "pred_ridge",
        "pred_rf",
        "pred_nn",
        "fold",
    }
    missing = expected_cols - set(predictions.columns)
    assert not missing, f"Loaded predictions missing columns: {missing}"

In [ ]:
print(predictions.shape)
predictions.head()

In [ ]:
# Section 5 sanity: schema + uniqueness

expected = {
    "Date",
    "PortfolioPair",
    "Actual",
    "pred_ridge",
    "pred_rf",
    "pred_nn",
    "fold",
}
missing = expected - set(predictions)
assert not missing, f"Missing columns in predictions: {missing}"

predictions["Date"] = pd.to_datetime(predictions["Date"])

# Each (Date, PortfolioPair) should appear once (test windows shouldn’t overlap).
dup_max = predictions.groupby(["Date", "PortfolioPair"], observed=False).size().max()
assert (
    dup_max == 1
), f"Duplicate (Date, PortfolioPair) rows found (max count={dup_max})."

print("predictions shape:", predictions.shape)
print(
    "date range:",
    predictions["Date"].min().date(),
    "→",
    predictions["Date"].max().date(),
)
print("n portfolios:", predictions["PortfolioPair"].nunique())

In [ ]:
# Section 5 sanity: cross-sectional coverage per test date

cs = predictions.groupby("Date")["PortfolioPair"].nunique()
display(cs.describe())

# Show a couple of smallest cross-sections (should usually be stable).
display(cs.sort_values().head(10))

</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  margin: 24px 0;
">

With walk-forward training complete, the notebook produces an out-of-sample prediction panel for each test date and portfolio. Each row corresponds to a unique (Date, Portfolio) observation and includes the realized next-period return (21 trading days, chosen to align the evaluation horizon with monthly portfolio implementation) together with model-implied signals from Ridge, Random Forest, the shallow neural network, and—where applicable—the boosted-tree (XGBoost) model.

The comparison is clean by construction: all models—including the boosted-tree extension where applicable—are trained on the same rolling windows and evaluated on the same held-out test observations. Scale-sensitive methods (Ridge and the neural network) use feature standardization fit within each training window and applied consistently to subsequent periods, preserving temporal ordering and avoiding look-ahead. The boosted-tree model is included as a robustness extension rather than a primary benchmark, allowing nonlinear sequential fitting behavior to be assessed under the same evaluation protocol.

At this stage, the outputs should be interpreted as **signals**, not portfolio performance. The key question is whether the model scores provide useful **cross-sectional ranking information** about next-period returns, before introducing any portfolio mechanics or trading frictions.

Accordingly, <a href="#section-6"><strong>Section 6</strong></a> shifts from model fitting to **signal diagnostics**. We evaluate the Information Coefficient (Spearman rank correlation) computed cross-sectionally at each test date, examine its stability over time with rolling summaries, and compare average IC levels across model classes.

Because the cross-sectional size is stable through time, changes in IC primarily reflect signal quality rather than variation in the evaluation universe. Mean-squared error is used as the training objective for numerical stability and comparability across model classes, while evaluation focuses on IC to align with the ranking-based decision problem faced in portfolio construction.

This section therefore evaluates signal quality in isolation—before any portfolio construction, rebalancing, or transaction cost assumptions are imposed.
</div>


---

<div id="section-6" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
">
    
**6. Signal Diagnostics**
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">

Before translating model outputs into portfolios, it is useful to examine whether the resulting signals appear **promising at the ranking level**. This section focuses on **cross-sectional signal diagnostics**, abstracting from trading frictions to assess how different models order portfolios over time.

The primary diagnostic is the **Information Coefficient (IC)**, defined as the cross-sectional Spearman rank correlation between predicted and realized returns in each evaluation period. IC provides a scale-free measure of ranking quality and is well suited to settings where absolute return prediction is noisy but relative ordering may still be informative. An IC of ~1% at the daily portfolio level corresponds to economically meaningful ranking information once aggregated through portfolio construction, even though it appears small in isolation.

Formal statistical inference on ICs (e.g., Newey–West adjusted t-statistics) is intentionally omitted, as the emphasis here is on economic behavior under realistic implementation rather than hypothesis testing.

<strong>Figure 3</strong> plots the **rolling IC time series** for each model. This visualization highlights both the average level of ranking skill and its stability through time. Periods of persistent positive IC suggest that a model is capturing structure that survives beyond short-term noise, while large swings or sign reversals indicate fragility.

To complement the time-series view, <strong>Figure 4</strong> summarizes **mean IC by model** over the full sample window. This comparison provides a compact snapshot of relative ranking performance once the expanded cross-section and higher frequency are taken into account.

These diagnostics are intentionally descriptive rather than inferential. The purpose is not to establish statistical dominance, but to answer a practical question: **do the signals generated by each model look sufficiently stable and directional to warrant further consideration?**

An alternative implementation could rebalance quarterly rather than monthly to further suppress turnover. Qualitative inspection suggests this would likely narrow performance dispersion across models without altering the qualitative ranking of linear versus nonlinear approaches.
</div>

In [ ]:
from src.harvard.signals import compute_signal_diagnostics

MODEL_COLS = {
    "Ridge": "pred_ridge",
    "RandomForest": "pred_rf",
    "NeuralNetwork": "pred_nn",
    "XGBoost": "pred_xgb",
}

artifacts = compute_signal_diagnostics(
    predictions=predictions,
    model_cols=MODEL_COLS,
    window=126,  # ~6 months trading days
)

In [ ]:
from src.harvard.plots import plot_mean_ic_bars, plot_rolling_ic_timeseries

plot_rolling_ic_timeseries(
    artifacts.ic_rolling,
    title="Rolling IC (≈6-Month Trailing Mean)",
)

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
"><strong>Figure 3: Rolling IC by Model</strong>
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  margin: 24px 0;
">Across the sample, 6-month rolling ICs typically fluctuate within a band of approximately −0.10 to +0.15, with occasional spikes during stress periods, while Ridge and the neural network exhibit comparatively smoother trajectories than tree-based models.
</div>

In [ ]:
plot_mean_ic_bars(artifacts.summary_table)

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
"><strong>Figure 4: Mean IC by Model</strong>
</div>

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  margin: 24px 0;
">At the daily portfolio level, mean IC values on the order of 0.7–1.0% are economically meaningful, corresponding to persistent but modest cross-sectional ranking skill consistent with prior factor-model evidence at higher frequency. 
Specifically, mean IC is approximately 1.00% for the neural network, 0.79% for Ridge, 0.71% for Random Forest, and 0.54% for XGBoost.
</div>

In [ ]:
artifacts.summary_table

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  margin: 24px 0;
">
Across the full sample, all models exhibit positive average Information Coefficients, indicating economically meaningful cross-sectional ranking information at the daily portfolio level. Ridge regression delivers the strongest and most stable signal, consistent with the persistence of linear factor structure in the FF100 universe. The shallow neural network produces ranking performance comparable to, and in this specification slightly exceeding, the Random Forest, suggesting that limited nonlinear structure can be captured by smooth, low-capacity transformations.
<br><br>
In contrast, the boosted-tree model exhibits a lower average IC than the other approaches once evaluated under the same walk-forward protocol and feature set. This outcome indicates that additional model flexibility does not mechanically translate into stronger ranking signals in this setting. Differences in mean IC across model classes are modest, reinforcing the view that while nonlinear effects are present, linear structure remains dominant in daily FF100 portfolio returns.
<br><br>
The Random Forest continues to provide a useful nonlinear benchmark, but does not consistently outperform the linear baseline in this setting. Taken together, these results suggest that gains from additional model flexibility are incremental rather than transformative, particularly at the daily portfolio level.
<br><br>

<strong>Interpreting IC magnitude across horizons.</strong>
<br>
The magnitude of the Information Coefficient depends on the evaluation horizon and does not scale linearly with frequency. At a daily horizon, realized returns are dominated by transitory noise, leading to mechanically smaller IC values even when underlying ranking information is present. Longer-horizon evaluations benefit from return aggregation and partial noise cancellation, which can amplify the measured strength of persistent signals. This perspective helps reconcile why increased observation counts and richer cross-sections do not necessarily translate into dominant daily IC gains for more flexible models.
<br><br>
Accordingly, daily IC values should be interpreted as indicators of <em>relative ranking stability</em> rather than as direct analogues of lower-frequency IC magnitudes. A model that exhibits modest but consistently positive daily IC can still encode economically meaningful structure, while additional data frequency does not guarantee proportional improvements in ranking performance.
<br><br>
The next section examines whether these differences in ranking performance translate into economically meaningful outcomes once the signals are implemented in long–short portfolios with realistic trading frictions.
</div>

---

<div id="section-7" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
">
  <strong>7. Portfolio Implementations</strong>
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">
    
Signal diagnostics alone do not determine whether a model is usable in practice. To assess economic relevance, predicted rankings are translated into **long–short portfolios**, allowing signal quality to be evaluated under realistic implementation constraints.

At each rebalance date, portfolios are ranked by their predicted returns. A **dollar-neutral long–short strategy** is constructed by going long the **top decile** of Fama–French portfolios and short the **bottom decile**, with equal weights assigned within each leg. Although signals are generated from daily data, portfolios are **rebalanced monthly**. This choice reflects a common practical compromise: daily signals can inform ranking stability, while monthly rebalancing limits excessive turnover and trading costs.

All models are evaluated using the same portfolio construction and rebalancing rules to ensure that differences in outcomes reflect signal behavior rather than implementation choices. Portfolio turnover is explicitly tracked by measuring changes in portfolio weights between rebalancing dates. To account for trading frictions, a **transaction cost of 10 basis points per unit of turnover** is applied. Net portfolio returns are computed by subtracting these costs from gross returns, allowing a direct comparison between signal strength and implementability.

<strong>Figure 5</strong> plots **cumulative portfolio returns** for each model, shown both before and after transaction costs. The divergence between gross and net performance across models provides a direct visualization of the interaction between signal stability and turnover, without relying on additional summary statistics.

This portfolio construction step is central to the case. Models that appear attractive based on ranking diagnostics may generate unstable signals that lead to high turnover and eroded returns once costs are applied. Conversely, models with more modest predictive strength can prove more effective if their signals are stable and inexpensive to trade. The analysis therefore shifts the focus from abstract predictive quality to **decision-relevant outcomes** under realistic constraints.
</div>

In [ ]:
from src.harvard.implementation import run_portfolio_implementations

# Build realized daily returns panel aligned to dataset.
returns = factor_panel[["Date", "Portfolio", "ExcessReturn"]].rename(
    columns={"ExcessReturn": "Return", "Portfolio": "PortfolioPair"}
)

port_artifacts = run_portfolio_implementations(
    predictions=predictions,
    returns=returns,
    model_cols=MODEL_COLS,
    rebalance="M",
    top_frac=0.10,  # Top/Bottom decile (FF100): long 10 portfolios, short 10 portfolios.
    cost_bps=10.0,  # <<< Cost assumption here.
)

ls_results_by_model = port_artifacts.ls_results_by_model
diagnostics_table = port_artifacts.diagnostics_table

In [ ]:
from src.harvard.plots import plot_portfolio_curves

# Plot gross vs net assuming 10 bps transaction costs.
plot_portfolio_curves(ls_results_by_model)

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
"><strong>Figure 5: Gross and Net Cumulative Returns by Model</strong>
</div>

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">

<a id="ref-e" href="#appendix-e"><strong>Appendix&nbsp;E</strong></a> reports net portfolio outcomes across a grid of transaction cost assumptions (5, 10, and 20 basis points), illustrating the sensitivity of each model’s results to trading frictions.

At a 10 bps transaction cost, Ridge retains close to break-even net performance with annualized turnover of roughly 8%, while nonlinear models exhibit materially higher turnover (≈16–19%), resulting in substantial erosion of net returns.
</div>

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  margin: 18px 0;
">

<strong>Interpretation.</strong>

Across models, the contrast between **gross** and **net** performance highlights the role of trading frictions in translating ranking signals into realized outcomes. While all models experience performance degradation once transaction costs are applied, the magnitude of this effect varies meaningfully with turnover and signal stability.

The linear baseline exhibits comparatively low turnover and more resilient net performance, despite only modest advantages in ranking metrics. Random Forest and neural network signals generate higher turnover, which erodes gross performance once costs are imposed and leads to weaker net outcomes over the full sample.

The boosted-tree model illustrates this trade-off most starkly. Although its signal-level diagnostics indicate persistent positive ranking information, higher turnover and signal instability result in inferior net portfolio performance across all transaction cost assumptions considered. This outcome reinforces the central message of the analysis: improvements in predictive metrics do not guarantee economic value when implementation frictions are taken into account.

Comparing these portfolio results to the signal-level diagnostics in <a href="#section-6"><strong>Section&nbsp;6</strong></a>, the <strong>relative ordering</strong> observed in Information Coefficients does not fully survive implementation. Although nonlinear models can exhibit competitive ranking performance in certain periods, differences in gross portfolio outcomes and signal stability are amplified once trading frictions are imposed. The linear baseline remains comparatively robust, combining moderate signal strength with low turnover.

Taken together, these results illustrate that economic relevance depends not only on ranking quality, but also on the consistency with which rankings translate into implementable portfolios. Portfolio implementation therefore acts as a stringent filter on model outputs, distinguishing between signals that are informative in isolation and those that remain effective under realistic trading constraints.

</div>


---

<div id="section-8" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
  line-height: 1.5;
  margin: 0 0 40px 0;
">
  <strong>8. Trade-offs and Practical Implications</strong>
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">

The portfolio results highlight several trade-offs that are not apparent from signal diagnostics alone. While ranking metrics such as the Information Coefficient provide a useful first screen, **implementation frictions materially shape realized outcomes**.

A key dimension is **turnover**. More flexible models tend to generate rankings that change more frequently, particularly in higher-frequency data. Even when such models exhibit positive and persistent ranking information, elevated turnover can erode returns once transaction costs are applied. In the present setting, this effect is most clearly illustrated by the boosted-tree model, which produces competitive signal diagnostics but underperforms economically due to higher trading intensity.

In contrast, more stable models may sacrifice some predictive sharpness but benefit from lower turnover and more resilient net performance. The linear baseline exemplifies this trade-off, combining modest ranking strength with implementation efficiency and robustness across transaction cost assumptions.

Another important consideration is **signal stability**. Cumulative return paths indicate that short bursts of strong performance are often followed by periods of degradation, particularly for models with unstable rankings and higher turnover. From a practical standpoint, strategies that deliver smoother performance profiles—even at the cost of lower peak returns—may be preferable to those that alternate between strong gains and rapid reversals.

Taken together, these results underscore that **model evaluation cannot be separated from portfolio construction**. Decisions about model complexity, data frequency, and rebalancing cadence jointly determine whether a signal remains usable once deployed. Improving predictive metrics in isolation does not guarantee better outcomes if those improvements come at the expense of higher turnover or instability.

For practitioners, the implication is clear: selecting a modeling approach requires balancing signal strength against implementation costs and robustness. The most effective models in this setting are not necessarily those that maximize ranking metrics, but those whose signals remain **stable, interpretable, and economical to trade**.

Because IC measures relative ranking rather than payoff magnitude, even stable positive IC does not guarantee attractive net portfolio returns once turnover and costs are introduced. This distinction becomes increasingly important as model complexity and signal responsiveness increase.
</div>


---

<div id="section-9" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
  line-height: 1.5;
  margin: 0 0 40px 0;
">
  <strong>9. Boosted Trees</strong>
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">

This section presents an optional robustness extension using boosted tree models and is not required for the main conclusions of the paper.

As an exploratory extension, this section briefly considers **boosted tree models** as an alternative nonlinear approach. While Random Forests rely on bagging—averaging many decorrelated trees—boosted trees build ensembles **sequentially**, with each tree focusing on residual errors from prior iterations. In applied settings, boosting is often viewed as a middle ground between linear models and more flexible architectures.

The boosted-tree implementation follows the same data, feature set, and walk-forward evaluation used throughout the case. Models are evaluated under the identical rolling training protocol, prediction horizon, and portfolio construction rules as the baseline specifications. Model complexity is deliberately constrained, using shallow tree depth and conservative learning rates to limit overfitting. The objective is to assess **behavioral differences**, not to optimize performance through extensive tuning.

When evaluated under this fully consistent framework, boosted trees do not outperform the baseline models. Although they produce positive ranking signals, their average Information Coefficient is lower than that of the linear and other nonlinear benchmarks, and their signals exhibit higher turnover. As a result, boosted trees generate inferior net portfolio performance once transaction costs are applied.

This outcome illustrates a broader point about model flexibility in high-frequency portfolio settings. Sequential fitting can amplify short-term responsiveness, but this responsiveness often manifests as unstable rankings and elevated trading intensity. As with other nonlinear models, any incremental gains in raw signal sharpness must be weighed against implementation costs and robustness.

Overall, this robustness check reinforces the central message of the case: **model class alone does not determine usability**. Even powerful nonlinear methods can underperform economically when increased flexibility leads to unstable signals or excessive trading. Boosted trees may warrant further exploration in alternative data regimes or with different objectives, but in the present setting they do not alter the trade-offs highlighted in the main analysis.
</div>

---

<div id="section-10" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
  line-height: 1.5;
  margin: 0 0 40px 0;
">
  <strong>10. Conclusion</strong>
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
">
    
<ul>
    <li>This case study highlights how conclusions drawn from machine-learning models depend critically on the data environment and on how signals are ultimately deployed in practice. Expanding both the cross-section and the frequency of factor portfolios improves the stability of ranking signals relative to small, monthly panels, but does not eliminate meaningful trade-offs across model classes.</li><br>
    <li>From a portfolio perspective, raw predictive metrics alone are insufficient guides to performance. Models that exhibit positive and persistent ranking information can nonetheless generate unstable signals and high turnover, eroding returns once realistic transaction costs are applied. Conversely, models with more modest ranking strength may deliver more robust net outcomes if their signals are stable and economical to trade.</li><br>
    <li>The central takeaway is practical rather than technical: model choice, data scale, and portfolio implementation must be considered jointly. In practice, improving signal quality, controlling turnover, and designing robust portfolio implementations are inseparable; progress on any single dimension is insufficient without coordination across the full pipeline. Increased model flexibility does not guarantee economic value, and in this setting, disciplined linear approaches remain competitive precisely because they balance signal strength with implementation robustness.</li><br>
    <li>For practitioners, the dominant risk is not insufficient model complexity, but excessive signal instability once turnover and costs are imposed.</li><br>
</ul>
</div>

---

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 30px;
  font-weight: 400;
  line-height: 1.5;
  margin: 0 0 40px 0;
"><strong>Appendices</strong>
</div>

---

<div id="appendix-a" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 26px;
  font-weight: 400;
  line-height: 1.5;
  margin: 0 0 40px 0;
">
  <strong>Appendix A</strong>
    <div style="
      font-family: 'Times New Roman', Times, serif;
      font-size: 20px;
      font-weight: 400;
      margin: 24px 0;
    "><a href="#ref-a"> [return]</a></div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  margin: 24px 0;
">

**Table 1: Fama--French Five Factors (MktRF, SMB, HML, RMW, CMA) plus the Momentum factor (UMD).**
    
  <table style="width:100%; border-collapse: collapse;">
    <thead>
      <tr>
        <th style="text-align:left; border-bottom:2px solid #000; padding:6px;">Factor</th>
        <th style="text-align:left; border-bottom:2px solid #000; padding:6px;">Name</th>
        <th style="text-align:left; border-bottom:2px solid #000; padding:6px;">Economic Rationale</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td style="padding:6px;">MktRF</td>
        <td style="padding:6px;">Market Excess Return</td>
        <td style="padding:6px;">
          Daily market return minus the risk-free rate. Investors require compensation for bearing
          broad, undiversifiable equity-market risk.
        </td>
      </tr>
      <tr>
        <td style="padding:6px;">SMB</td>
        <td style="padding:6px;">Small Minus Big</td>
        <td style="padding:6px;">
          Small-cap firms are riskier—less diversified and more financially constrained—and therefore
          earn higher expected returns.
        </td>
      </tr>
      <tr>
        <td style="padding:6px;">HML</td>
        <td style="padding:6px;">High Minus Low (Value)</td>
        <td style="padding:6px;">
          High book-to-market firms tend to be distressed or out-of-favor; they earn premia for
          bearing additional fundamental risk.
        </td>
      </tr>
      <tr>
        <td style="padding:6px;">RMW</td>
        <td style="padding:6px;">Robust Minus Weak (Profitability)</td>
        <td style="padding:6px;">
          Highly profitable firms are more resilient and earn higher expected returns; low-profitability
          firms are riskier and command a discount.
        </td>
      </tr>
      <tr>
        <td style="padding:6px;">CMA</td>
        <td style="padding:6px;">Conservative Minus Aggressive (Investment)</td>
        <td style="padding:6px;">
          Firms that invest conservatively (low asset growth) tend to outperform aggressively
          investing firms, which often overinvest in low NPV projects.
        </td>
      </tr>
      <tr>
        <td style="padding:6px;">UMD</td>
        <td style="padding:6px;">Momentum</td>
        <td style="padding:6px;">
          Assets with strong past returns tend to continue outperforming in the medium term,
          reflecting momentum in returns.
        </td>
      </tr>
    </tbody>
  </table>
</div>

---

<div id="appendix-b" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 26px;
  font-weight: 400;
  line-height: 1.5;
  margin: 0 0 40px 0;
">
  <strong>Appendix B</strong>
    <div style="
      font-family: 'Times New Roman', Times, serif;
      font-size: 20px;
      font-weight: 400;
      margin: 24px 0;
    "><a href="#ref-b"> [return]</a></div>
</div>

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  margin: 24px 0;
">

<strong>Distributional Properties of FF100 Portfolio Returns</strong>
<br>
To characterize the empirical environment in which models are trained and evaluated,
Table&nbsp;B.1 first summarizes the cross-portfolio distribution of key return statistics
across the 100 Fama–French Size × Book-to-Market portfolios. For each statistic—number of
observations, mean, standard deviation, minimum, maximum, skewness, and excess kurtosis—we
report the minimum, maximum, mean, and standard deviation across portfolios.

These summary statistics highlight substantial cross-sectional heterogeneity and
pronounced deviations from normality. In particular, many portfolios exhibit negative
skewness and large excess kurtosis, indicating fat-tailed return distributions and
asymmetric downside risk. Such distributional properties help explain why unconditional
mean returns provide a limited guide to relative performance and motivate a
ranking-based evaluation framework.

Table&nbsp;B.2 then reports the corresponding statistics at the individual portfolio level.
To aid readability, the table is presented in two blocks, listing the first 50 and last
50 portfolios. Together, these tables document the dispersion and tail behavior present
across the full FF100 cross-section and provide empirical context for the modeling and
portfolio implementation results discussed in the main text.

</div>

In [ ]:
from src.harvard.diagnostics import (
    compute_portfolio_return_stats,
    compute_portfolio_return_stats_summary,
)

pd.options.display.float_format = "{:0.6f}".format

portfolio_stats = compute_portfolio_return_stats(portfolio_returns)

portfolio_stats.to_csv(
    "../results/ff100_portfolio_return_stats.csv",
    index=False,
)
table_b1 = compute_portfolio_return_stats_summary(portfolio_stats)

# Table B.1: Summary stats of portfolio returns (100 portfolios)
table_b1

In [ ]:
# Table B.2: Portfolio returns (100 portfolios, ME01-ME05)

portfolio_stats.head(50)

In [ ]:
# Table B.2 (cont.): Portfolio returns (100 portfolios, ME06-ME10)

portfolio_stats.tail(50)

---

<div id="appendix-c" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 26px;
  font-weight: 400;
  line-height: 1.5;
  margin: 0 0 40px 0;
">
  <strong>Appendix C</strong>
    <div style="
      font-family: 'Times New Roman', Times, serif;
      font-size: 20px;
      font-weight: 400;
      margin: 24px 0;
    "><a href="#ref-c"> [return]</a></div>
</div>
<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  margin: 24px 0;
">

**Table 2: Daily feature construction.**
<br>
*All rolling statistics and lags are computed causally using past data only.<br>
Lag sets are written compactly (e.g., SMB_lag{21,63,126,252}).<br>
Rolling windows use a fixed 252 trading day horizon (12-month).*

<table style="width:100%; border-collapse: collapse;">
    <thead>
      <tr>
        <th style="text-align:left; border-bottom:2px solid #000; padding:6px;">Category</th>
        <th style="text-align:left; border-bottom:2px solid #000; padding:6px;">Feature(s)</th>
        <th style="text-align:left; border-bottom:2px solid #000; padding:6px;">Definition</th>
      </tr>
    </thead>
    <tbody>

  <tr>
    <td style="padding:6px; font-weight:600;">Risk-Free &amp; Factor Returns</td>
    <td style="padding:6px;">RF, MktRF, SMB, HML, RMW, CMA, UMD</td>
    <td style="padding:6px;">
      Daily risk-free rate and daily Fama–French factor returns used as base inputs for feature
      construction.
    </td>
  </tr>

  <tr>
    <td style="padding:6px; font-weight:600;">Smoothed Factor Levels</td>
    <td style="padding:6px;">{Factor}_ma21</td>
    <td style="padding:6px;">
      21-day moving average of daily factor returns, used to reduce high-frequency noise while
      preserving short-term dynamics.
    </td>
  </tr>

  <tr>
    <td style="padding:6px; font-weight:600;">Lagged Factor Returns</td>
    <td style="padding:6px;">{Factor}_lag21, lag63, lag126, lag252</td>
    <td style="padding:6px;">
      Factor returns lagged by 21, 63, 126, and 252 trading days, corresponding approximately to
      1-, 3-, 6-, and 12-month horizons.
    </td>
  </tr>

  <tr>
    <td style="padding:6px; font-weight:600;">Rolling Factor Statistics</td>
    <td style="padding:6px;">{Factor}_roll252_mean, {Factor}_roll252_std</td>
    <td style="padding:6px;">
      Trailing 252-day (12-month) arithmetic mean and standard deviation of daily factor returns,
      capturing long-run level and volatility.
    </td>
  </tr>

  <tr>
    <td style="padding:6px; font-weight:600;">Portfolio Return Features</td>
    <td style="padding:6px;">
      pf_ret_lag21, lag63, lag126, lag252;<br>
      pf_ret_roll252_mean, pf_ret_roll252_std
    </td>
    <td style="padding:6px;">
      Portfolio-specific return lags and rolling statistics computed using daily returns and
      trading-day equivalents of monthly horizons.
    </td>
  </tr>

  <tr>
    <td style="padding:6px; font-weight:600;">Portfolio Momentum (12–1)</td>
    <td style="padding:6px;">pf_mom_252_21</td>
    <td style="padding:6px;">
      Momentum computed as the cumulative portfolio return over the 252-day window ending at
      t−21, excluding the most recent 21 trading days. This mirrors the standard 12–1 momentum
      definition in daily time.
    </td>
  </tr>

</tbody>
  </table>
</div>

</div>

In [ ]:
factor_panel.columns

---

<div id="appendix-d" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 26px;
  font-weight: 400;
  line-height: 1.5;
  margin: 0 0 40px 0;
">
  <strong>Appendix D</strong>
    <div style="
      font-family: 'Times New Roman', Times, serif;
      font-size: 20px;
      font-weight: 400;
      margin: 24px 0;
    "><a href="#ref-d"> [return]</a></div>
</div>

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  margin: 24px 0;
">

<strong>Model Architecture, Training, and Evaluation Details</strong>

This appendix provides detailed documentation of the model architectures, optimization
procedures, and evaluation choices used in the analysis. The emphasis is on transparency
and reproducibility rather than exhaustive hyperparameter tuning. All models are trained
and evaluated within the rolling walk-forward framework described in
<a href="#section-4">Section&nbsp;4</a>, with model specifications and design choices
introduced in <a href="#section-5">Section&nbsp;5</a>.

<strong>Data Frequency and Prediction Task</strong>

<ul style="margin-top:8px;">
  <li><strong>Frequency:</strong> Daily observations.</li>
  <li><strong>Cross-section:</strong> Fama–French 100 Size × Book-to-Market portfolios.</li>
  <li><strong>Prediction target:</strong> Next-period portfolio return (21 trading days).</li>
  <li><strong>Evaluation focus:</strong> Cross-sectional ranking performance rather than level accuracy.</li>
</ul>

The FF100 universe provides a substantially richer cross-section than the FF25 panel.
This increased dispersion improves the detectability of ranking signals while retaining economic
interpretability through characteristic-sorted portfolios.

<strong>Walk-Forward Protocol (All Models)</strong>

<ul style="margin-top:8px;">
  <li><strong>Splits:</strong> train ≈ 1134 trading days, val 126, purge 21, test 21, step 21.</li>
  <li><strong>Temporal ordering:</strong> Strictly preserved to avoid look-ahead bias.</li>
  <li><strong>Data leakage control:</strong> All preprocessing and scaling recomputed within each training window using past data only.</li>
  <li><strong>Training objective:</strong> Mean-squared error (chosen for numerical stability).</li>
</ul>

<strong>Linear Baseline: Ridge Regression</strong>

<ul style="margin-top:8px;">
  <li><strong>Model:</strong> Linear regression with ℓ<sub>2</sub> regularization (Ridge).</li>
  <li><strong>Loss function:</strong> Mean-squared error (MSE).</li>
  <li><strong>Regularization:</strong> λ ∈ {10<sup>−4</sup>, 10<sup>−3</sup>, 10<sup>−2</sup>, 10<sup>−1</sup>, 1}, selected per fold on the validation window; refit on train+val.</li>
  <li><strong>Feature scaling:</strong> Standardization (zero mean, unit variance) fit within each training window and applied to validation and test periods.</li>
  <li><strong>Motivation:</strong> Ridge stabilizes estimation under multicollinearity while preserving interpretability in a factor-style linear framework.</li>
  <li><strong>Role in analysis:</strong> Primary linear benchmark for assessing whether nonlinear models deliver incremental ranking value.</li>
</ul>

<strong>Nonlinear Benchmark: Random Forest</strong>

<ul style="margin-top:8px;">
  <li><strong>Model:</strong> Random Forest regressor composed of an ensemble of decision trees.</li>
  <li><strong>Loss function:</strong> Mean-squared error (MSE).</li>
  <li><strong>Hyperparameters:</strong>
    <ul style="margin-top:4px;">
      <li>Grid search over:
        <ul style="margin-top:4px;">
          <li>n<sub>estimators</sub> ∈ {100}</li>
          <li>max_depth ∈ {5, 10, None}</li>
          <li>min_samples_split ∈ {10, 20}</li>
          <li>max_features ∈ {"sqrt", "log2"}</li>
        </ul>
        Selected per fold on validation IC; best config refit on train+val.
      </li>
    </ul>
  </li>
  <li><strong>Subsampling:</strong> Bootstrap sampling of observations and random feature selection at each split.</li>
  <li><strong>Feature scaling:</strong> None (tree-based methods are scale-invariant).</li>
  <li><strong>Motivation:</strong> Capture nonlinear interactions and conditional structure among factor lags, rolling statistics, and portfolio characteristics.</li>
  <li><strong>Role in analysis:</strong> Primary nonlinear reference model.</li>
</ul>

<strong>Sensitivity Check: Shallow Neural Network</strong>

<ul style="margin-top:8px;">
  <li><strong>Model:</strong> Fully connected multilayer perceptron (MLP).</li>
  <li><strong>Architecture:</strong> Input → 64 → 32 → 1 (three layers, two hidden).</li>
  <li><strong>Activation:</strong> ReLU hidden layers; linear output.</li>
  <li><strong>Dropout:</strong> 0.1 applied to hidden layers during training.</li>
  <li><strong>Loss function:</strong> Mean-squared error (MSE).</li>
  <li><strong>Optimization:</strong> Mini-batch SGD with validation-driven early stopping (patience) to mirror the baseline NN.</li>
  <li><strong>Feature scaling:</strong> Standardization fit within each training window.</li>
  <li><strong>Motivation:</strong> Flexible function class; sensitive to scale and objective mismatch, so interpreted as a sensitivity check.</li>
  <li><strong>Role in analysis:</strong> Illustrative, not the primary contender.</li>
</ul>

<strong>Nonlinear Extension: Boosted Trees (XGBoost)</strong>
<ul style="margin-top:8px;"> 
    <li><strong>Model:</strong> Gradient-boosted decision tree ensemble implemented using XGBoost.</li> 
    <li><strong>Loss function:</strong> Mean-squared error (MSE), consistent with other model classes.</li> 
    <li><strong>Training protocol:</strong> Integrated directly into the same rolling walk-forward framework used for Ridge, Random Forest, and the neural network. Boosted trees are trained, validated, and evaluated on identical train–validation–test splits, with a purge buffer to prevent temporal leakage.</li> 
    <li><strong>Hyperparameter selection:</strong> 
        <ul style="margin-top:4px;"> 
            <li>Constrained grid search on the validation window using mean daily Information Coefficient (IC) as the selection criterion.</li> 
            <li>Representative parameter ranges: 
                <ul style="margin-top:4px;"> 
                    <li>max_depth ∈ {2, 3}</li> 
                    <li>learning_rate ∈ {0.03, 0.05}</li> 
                    <li>n_estimators ∈ {300, 600}</li> 
                    <li>min_child_weight ∈ {5, 10}</li> 
                    <li>reg_lambda ∈ {1.0, 5.0}</li> 
                </ul> </li> 
            <li>The selected configuration is refit on the combined train+validation window before generating test-period predictions.</li> 
        </ul> 
    </li> 
    <li><strong>Feature scaling:</strong> None (tree-based methods are scale-invariant).</li> 
    <li><strong>Model complexity controls:</strong> Shallow tree depth, conservative learning rates, and no early stopping are used to limit overfitting and ensure comparability with other models.</li> 
    <li><strong>Parallel execution:</strong> When evaluated in the parallel walk-forward pipeline, boosted trees are run single-threaded within each worker to avoid oversubscription and to preserve deterministic behavior.</li> 
    <li><strong>Motivation:</strong> Boosted trees provide a complementary nonlinear benchmark within the tree-ensemble family, emphasizing sequential fitting rather than bagging.</li> 
    <li><strong>Role in analysis:</strong> Optional robustness check. Results are interpreted as evidence about the stability and implementability of more flexible nonlinear models, rather than as an attempt to optimize boosted-tree performance.</li> 
</ul>

<strong>Evaluation Metric and Objective Mismatch</strong>

<ul style="margin-top:8px;">
  <li><strong>Primary evaluation metric:</strong> Information Coefficient (IC), defined as the cross-sectional Spearman rank correlation between predicted and realized returns.</li>
  <li><strong>Rationale:</strong> In noisy daily return settings, absolute level prediction is unstable, while relative ordering may still be informative.</li>
  <li><strong>Objective mismatch:</strong> Models train on MSE but are evaluated on a rank-based metric; this can influence relative behavior, especially for neural networks, and motivates cautious interpretation of NN results.</li>
</ul>

<strong>Interpretation Scope</strong>

<ul style="margin-top:8px;">
  <li>No statistical dominance claims are made at this stage.</li>
  <li>Diagnostics focus on signal directionality, stability, and relative ordering.</li>
  <li>Economic implementation and trading frictions are deferred to Section&nbsp;7.</li>
</ul>

</div>


---

<div id="appendix-e" style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 26px;
  font-weight: 400;
  line-height: 1.5;
  margin: 0 0 40px 0;
">
  <strong>Appendix E</strong>
    <div style="
      font-family: 'Times New Roman', Times, serif;
      font-size: 20px;
      font-weight: 400;
      margin: 24px 0;
    "><a href="#ref-e"> [return]</a></div>
</div>

<div style="
  font-family: 'Times New Roman', Times, serif;
  font-size: 20px;
  font-weight: 400;
  margin: 24px 0;
">

<strong>Transaction Cost Sensitivity</strong>

Net portfolio outcomes across a grid of transaction cost assumptions (5, 10, and 20 basis points).

In [ ]:
from src.harvard.portfolio import build_cost_grid_table

cost_table = (
    build_cost_grid_table(
        predictions=predictions,
        model_cols=MODEL_COLS,
        cost_grid_bps=(5.0, 10.0, 20.0),
    )
    .sort_values(["model", "cost_bps"])
    .reset_index(drop=True)
)

cost_table.round(
    {
        "ann_return": 4,
        "ann_vol": 4,
        "sharpe": 3,
        "max_drawdown": 3,
        "avg_turnover": 3,
        "ann_turnover": 1,
    }
)